## Vidushi

In [36]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize

# Decision Tree

*Imbalanced Data* 

In [77]:
data = pd.read_csv('data/processedData.csv')
smote_data= pd.read_csv('data/processedDataSMOTE.csv')

In [76]:
data.isna().sum()

Unnamed: 0                       0
age                              0
duration                         0
campaign                         0
pdays                            0
previous                         0
emp.var.rate                     0
cons.price.idx                   0
cons.conf.idx                    0
euribor3m                        0
nr.employed                      0
y                                0
job_admin.                       0
job_blue-collar                  0
job_entrepreneur                 0
job_housemaid                    0
job_management                   0
job_retired                      0
job_self-employed                0
job_services                     0
job_student                      0
job_technician                   0
job_unemployed                   0
marital_divorced                 0
marital_married                  0
marital_single                   0
education_basic.4y               0
education_basic.6y               0
education_basic.9y  

In [48]:
data.y.replace(to_replace = ['yes','no'], value = [1,0],inplace = True)

In [49]:
y=data['y']
X=data.drop(columns=['y'])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [51]:
tree = DecisionTreeClassifier()

In [52]:
model = tree.fit(X_train,y_train)

In [53]:
y_pred = model.predict(X_test)

In [54]:
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))

Precision:  0.52 Recall:  0.54


In [73]:
param_grid = {'criterion': ['gini','entropy'],
              'min_samples_split': [10,20,30],
              'max_depth': [1,5,10, 20, 25, 30],
              'min_samples_leaf': [5, 10,15,20],
              'max_leaf_nodes': [5, 10, 20]}
grid = GridSearchCV(tree, param_grid, cv=6, scoring='recall')
grid.fit(X_train, y_train)

GridSearchCV(cv=6, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=5,
                                              max_features=None,
                                              max_leaf_nodes=10,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                        

In [58]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_leaf_nodes': 10,
 'min_samples_leaf': 5,
 'min_samples_split': 10}

In [69]:
tree_opt = DecisionTreeClassifier(criterion='gini', max_depth=5,max_leaf_nodes = 10)

In [70]:
model_opt = tree_opt.fit(X_train,y_train)

In [71]:
y_pred_opt = model_opt.predict(X_test)

In [72]:
print("Precision: ",round(precision_score(y_test,y_pred_opt),2),"Recall: ",round(recall_score(y_test,y_pred_opt),2))

Precision:  0.58 Recall:  0.66


In [78]:
smote_data.isna().sum()

Unnamed: 0                           0
age                              22842
duration                             0
campaign                             0
pdays                                0
previous                             0
emp.var.rate                         0
cons.price.idx                       0
cons.conf.idx                        0
euribor3m                            0
nr.employed                          0
job_admin.                           0
job_blue-collar                      0
job_entrepreneur                     0
job_housemaid                        0
job_management                       0
job_retired                          0
job_self-employed                    0
job_services                         0
job_student                          0
job_technician                       0
job_unemployed                       0
marital_divorced                     0
marital_married                      0
marital_single                       0
education_basic.4y       